[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainXGBmodel.ipynb)

In [1]:
!pip install xgboost

In [0]:
import xgboost as xgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/
%ls

/content/gdrive/My Drive/projects
model_LGB.sav  user_train_wip.csv


In [0]:
s_data = 'user_train_wip.csv'
s_col_tgt = 'totals.transactionRevenue_sum'
l_col_drop = ['date_min', 'date_max',s_col_tgt]
filename = 'model_XGB.sav'

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 11.5 s, sys: 2.64 s, total: 14.1 s
Wall time: 16.1 s


In [7]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)


CPU times: user 2.24 s, sys: 593 ms, total: 2.83 s
Wall time: 2.83 s


In [0]:
params = {
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": st.beta(10, 1),
    "subsample": st.beta(10, 1),
    "gamma": st.uniform(0, 10),
    'reg_alpha': st.expon(0, 50),
    "min_child_weight": st.expon(0, 50),
}

model = xgb.XGBRegressor(**params)

In [0]:
model = xgb.XGBRegressor()

In [12]:
%%time
model.fit(x_train,y_train)

CPU times: user 2min 23s, sys: 387 ms, total: 2min 24s
Wall time: 2min 24s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [13]:
%%time
pickle.dump(model, open(filename, 'wb'))

CPU times: user 969 µs, sys: 1.04 ms, total: 2.01 ms
Wall time: 10.9 ms


In [0]:
files.download(filename) 